In [ ]:
from google.colab import auth
from google.auth import default
from google.auth.transport.requests import Request
import gspread
import pandas as pd
import re
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, Border, Side

# 1. Đăng nhập vào tài khoản Google của bạn
auth.authenticate_user()

# 2. Sử dụng thông tin xác thực mặc định của Colab
creds, _ = default()

# 3. Làm mới thông tin xác thực nếu cần
creds.refresh(Request())

# 4. Kết nối với Google Sheets
gc = gspread.authorize(creds)

try:
    # 5. Mở Google Sheet bằng URL
    sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1rdcNBfwBGkeMxrpGWo77I8Eyi1_3H9-57XQNOsJ-_cM/edit?gid=1968705733')

    # 6. Chọn trang tính (worksheet) đầu tiên
    worksheet = sheet.get_worksheet(0)

    # 7. Lấy tất cả dữ liệu từ trang tính
    all_data = worksheet.get_all_values()

    if not all_data:
        raise ValueError("Không có dữ liệu nào được lấy từ Google Sheets. Vui lòng kiểm tra quyền truy cập và URL.")

    # 8. Chuyển dữ liệu thành DataFrame
    df = pd.DataFrame(all_data[1:], columns=all_data[0])

    # 9. Loại bỏ khoảng trắng dư thừa trong tên cột
    df.columns = df.columns.str.strip()

    # Làm sạch dữ liệu cột "Dấu thời gian"
    df['Dấu thời gian'] = df['Dấu thời gian'].str.strip()  # Xóa khoảng trắng dư thừa
    df['Dấu thời gian'] = df['Dấu thời gian'].replace('', pd.NA)  # Thay thế chuỗi rỗng bằng NA

    # 10. Chuyển đổi số liệu trong các cột cụ thể
    def convert_to_int(value):
        value = re.sub(r'\D', '', value)  # Loại bỏ ký tự không phải số
        return int(value) if value.isdigit() else 0

    df['Số tiết thực giảng'] = df['Số tiết thực giảng'].apply(convert_to_int)
    df['Số lượng HSSV có mặt'] = df['Số lượng HSSV có mặt'].apply(convert_to_int)
    df['Số lượng HSSV Vắng - Có lý do'] = df['Số lượng HSSV Vắng - Có lý do'].apply(convert_to_int)
    df['Số lượng HSSV Vắng - Không có lý do'] = df['Số lượng HSSV Vắng - Không có lý do'].apply(convert_to_int)

    # 11. Chuyển đổi cột "Dấu thời gian" sang định dạng ngày tháng linh hoạt
    df['Dấu thời gian'] = pd.to_datetime(df['Dấu thời gian'], errors='coerce', dayfirst=True)

    # 12. Lọc dữ liệu từ ngày cần lọc hết tháng đó
    df_august = df[(df['Dấu thời gian'].dt.month == 8) &
                   (df['Dấu thời gian'].dt.year == 2024) &
                   (df['Dấu thời gian'].dt.day >= 1)].copy()

    # 13. Thêm cột "Ngày" để theo dõi ngày của từng bản ghi
    df_august['Ngày'] = df_august['Dấu thời gian'].dt.strftime('%d/%m/%Y')

    # 14. Thêm cột "Buổi"
    def determine_session(time_obj):
        try:
            hour = time_obj.hour  # Lấy giờ từ đối tượng datetime
            if 7 <= hour <= 11:
                return 'Sáng'
            else:
                return 'Chiều'
        except AttributeError:
            return 'Không xác định'

    df_august['Buổi'] = df_august['Dấu thời gian'].apply(determine_session)

    # 15. Thêm cột số thứ tự (STT) và sắp xếp lại theo ngày và buổi
    df_august = df_august.sort_values(by=['Ngày', 'Buổi'])
    df_august.insert(0, 'STT', range(1, len(df_august) + 1))

    # 16. Sắp xếp các cột theo yêu cầu và thêm cột "Cán bộ chấm" đầu tiên và "Ghi chú" cuối cùng
    report_columns = [
        'STT',
        'Ngày',
        'Cán bộ chấm',
        'Họ tên giáo viên',
        'Môn giảng theo TKB',
        'Lớp',
        'Buổi',
        'Số tiết thực giảng',
        'Số lượng HSSV có mặt',
        'Số lượng HSSV Vắng - Có lý do',
        'Số lượng HSSV Vắng - Không có lý do',
        'Ghi chú'
    ]
    monthly_report = df_august[report_columns]

    # 17. Tạo workbook mới và sheet
    wb = Workbook()
    ws = wb.active

    # 18. Thêm tiêu đề báo cáo (in hoa)
    ws.merge_cells('A1:L1')
    ws['A1'] = 'BÁO CÁO THÁNG 8/2024'.upper()
    ws['A1'].font = Font(size=14, bold=True)
    ws['A1'].alignment = Alignment(horizontal='center')

    # 19. Thêm dữ liệu từ DataFrame vào Excel và định dạng bảng
    thin_border = Border(left=Side(style='thin'),
                         right=Side(style='thin'),
                         top=Side(style='thin'),
                         bottom=Side(style='thin'))

    for r_idx, row in enumerate(dataframe_to_rows(monthly_report, index=False, header=True), 4):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center')
            if r_idx == 4:  # Định dạng tiêu đề cột
                cell.font = Font(bold=True)

    # 20. Định dạng cột và dòng
    for col in ws.iter_cols(min_row=4, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        max_length = 0
        column = col[0].column_letter  # Lấy chữ cái của cột
        for cell in col:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        adjusted_width = max_length + 2
        ws.column_dimensions[column].width = adjusted_width

    # 21. Thêm ghi chú cuối bảng
    last_row = ws.max_row + 2
    ws.merge_cells(f'A{last_row}:L{last_row}')
    ws[f'A{last_row}'] = "PHÒNG THANH TRA - CÔNG TÁC HSSV".upper()
    ws[f'A{last_row}'].font = Font(bold=True)
    ws[f'A{last_row}'].alignment = Alignment(horizontal='right')

    # 22. Lưu file Excel vào thư mục hiện tại với tháng trong tên file
    file_name = 'Báo cáo tháng 8_2024.xlsx'
    wb.save(file_name)

    # Lưu dữ liệu thành file CSV
    csv_file_name = 'Báo cáo tháng 8_2024.csv'
    monthly_report.to_csv(csv_file_name, index=False)

    # Thông báo về kết quả lưu file
    print(f"Báo cáo tháng 8/2024 đã được lưu thành file Excel: {file_name}")
    print(f"Báo cáo tháng 8/2024 đã được lưu thành file CSV: {csv_file_name}")

except Exception as e:
    print(f"Lỗi khi truy cập Google Sheets: {e}")



Báo cáo tháng 8/2024 đã được lưu thành file Excel: Báo cáo tháng 8_2024.xlsx
Báo cáo tháng 8/2024 đã được lưu thành file CSV: Báo cáo tháng 8_2024.csv
